In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=gemini_api_key)

In [14]:
# query = "현재 빌보드 차트 1위 곡은?"
# query = "2025년 KT 소액결제 해킹 사건의 원인은?"
query = "2025년 KT 소액결제 해킹 사건 원인"

In [15]:
response = llm.invoke(query)
response

AIMessage(content='2025년에 KT 소액결제 해킹 사건이 실제로 발생했는지 확인이 필요합니다. 현재까지 공식적으로 발표된 내용은 없습니다.\n\n만약 미래에 KT 소액결제 해킹 사건이 발생한다면, 다음과 같은 원인들을 추정해 볼 수 있습니다:\n\n**기술적 원인:**\n\n*   **취약점 악용:** KT 시스템 또는 관련 API의 보안 취약점을 해커가 발견하고 악용하여 결제 시스템에 침투할 수 있습니다.\n*   **멀웨어 감염:** KT 내부 시스템이나 사용자 기기가 악성코드에 감염되어 결제 정보가 유출될 수 있습니다.\n*   **데이터베이스 침해:** KT의 결제 관련 데이터베이스가 해킹되어 개인 정보 및 결제 정보가 유출될 수 있습니다.\n*   **API 보안 미흡:** 외부 서비스와의 연동 과정에서 API 보안이 취약하여 해커가 결제 시스템에 접근할 수 있습니다.\n*   **인증 및 접근 통제 미흡:** 사용자 인증 절차가 허술하거나 접근 통제가 제대로 이루어지지 않아 해커가 권한을 획득할 수 있습니다.\n\n**관리적 원인:**\n\n*   **보안 투자 부족:** KT가 정보 보안에 충분한 투자를 하지 않아 시스템 보안 수준이 낮을 수 있습니다.\n*   **보안 교육 부족:** KT 직원들의 보안 의식 부족 및 보안 교육 미흡으로 인해 보안 사고가 발생할 수 있습니다.\n*   **보안 정책 미흡:** KT의 보안 정책이 미흡하거나 제대로 시행되지 않아 보안 사고를 예방하지 못할 수 있습니다.\n*   **협력업체 관리 부실:** KT와 협력하는 업체의 보안 관리가 허술하여 해킹 경로로 이용될 수 있습니다.\n*   **사이버 공격 대응 미흡:** 사이버 공격 발생 시 신속하고 효과적인 대응 체계가 갖춰져 있지 않아 피해가 확산될 수 있습니다.\n\n**사회공학적 원인:**\n\n*   **피싱/스미싱:** 해커가 KT 고객을 속여 개인 정보나 결제 정보를 탈취할 수 있습니다.\n*   **내부자 소행:** KT 내부 직원

In [16]:
from langchain_community.tools import DuckDuckGoSearchResults 

search = DuckDuckGoSearchResults(results_separator=';\n')
docs = search.invoke(query)

print(docs)

snippet: 10분 전 — 2025년 8월부터 수도권 일부 지역에서 불법 초소형 기지국을 이용해 KT와 KT망 알뜰폰 등 KT 이동통신망 이용 고객의 휴대전화에서 이용한 적 없는 휴대폰 소액결제가 이 ..., title: KT 고객 무단 소액결제 침해 사고, link: https://namu.wiki/w/KT+고객+무단+소액결제+침해+사고;
snippet: 2025. 9. 11. — 왜 KT만 털렸나 무단 소액결제 사건 의문점 3가지 KT는 11일 불법 초소형 기지국을 통해 일부 고객의 국제이동가입자식별정보IMSI 정보가 유출된 것 을 확인하고 이를 ..., title: 왜 KT만 털렸나… '무단 소액결제' 사건 의문점 3가지, link: https://www.chosun.com/economy/tech_it/2025/09/11/P6IGLCLDPFFARKYDTBAPSIHFYA/;
snippet: 6시간 전 — 2025년 9월 17일, 경기남부경찰청은 KT 무단 소액결제 사건 의 용의자로 조선족 2명을 긴급체포했다는 기사가 나왔다. . 이들은 불법 소형 기지국 장비를 승합차에 싣고 ..., title: KT 소액결제 범인 체포 (feat 애플계정과 롯데카드 추가로 털림), link: https://blog.naver.com/ranto28/224011762849;
snippet: 2025. 9. 11. — 사건 초기 KT 는 개인정보 해킹 정황은 없다고 발표했지만, 불법 초소형 기지국을 통한 개인정보 유출 가능성이 있는 정황을 확인하고, 11일 오후 개인정보보호위원회(이하 ..., title: KT 소액결제 사고, 무엇이 밝혀졌나…현재 대응 방법은? - IT동아, link: https://it.donga.com/107553/


In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "사용자의 질문에 대해 아래 context에 기반하여 답변하라.:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = question_answering_prompt | llm

In [9]:
from langchain.memory import ChatMessageHistory

# 채팅 메시지를 저장할 메모리 객체 생성
chat_history = ChatMessageHistory() 
# 사용자 질문을 메모리에 저장
chat_history.add_user_message(query)

# 문서 검색하고 답변 생성
answer = document_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": docs, # 검색 결과 전달
    }
)

# 생성된 답변을 메모리에 저장
chat_history.add_ai_message(answer) 

print(answer)

content="2025년 KT 소액결제 해킹 사건의 핵심 원인은 불법 초소형 기지국, 즉 '유령 기지국'으로 밝혀졌습니다. 해커들이 가상의 기지국을 만들어 개인정보를 빼돌리려 시도한 정황이 포착되었고, 단말이 기지국에 위치 정보를 등록하는 과정에서 국제이동가입자식별번호(IMSI)가 외부로 유출된 정황도 확인되었습니다." additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--09fab0a5-b7e1-46ed-912e-091b99d1bbec-0' usage_metadata={'input_tokens': 644, 'output_tokens': 121, 'total_tokens': 765, 'input_token_details': {'cache_read': 0}}


In [10]:
from langchain_core.tools import tool
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

@tool
def get_web_search(query: str, search_period: str='w') -> str:
    """
    웹 검색을 수행하는 함수.

    Args:
        query (str): 검색어
        search_period (str): 검색 기간 (e.g., "w" for past week (default), "m" for past month, "y" for past year, "d" for past day)

    Returns:
        str: 검색 결과
    """
    wrapper = DuckDuckGoSearchAPIWrapper(
        region="kr-kr",
        time=search_period
    )

    print('\n----- WEB SEARCH -----')
    print(query)
    print(search_period)

    search = DuckDuckGoSearchResults(
        api_wrapper=wrapper,
        source="news",
        results_separator=';\n'
    )

    searched = search.invoke(query)
    
    for i, result in enumerate(searched.split(';\n')):
        print(f'{i+1}. {result}')
    
    return searched

In [11]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

# 도구들을 tools 리스트에 추가
tools = [get_web_search] # 에이전트가 사용할 도구 리스트

# 에이전트 프롬프트 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    ("human", "{input}"),
    # AgentExecutor는 agent_scratchpad를 자동으로 채워 넣어,
    # 에이전트가 이전 단계의 행동과 결과를 기억하고 다음 행동을 결정할 수 있도록 함
    ("placeholder", "{agent_scratchpad}"),
])

# 에이전트 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 초기화
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [17]:
from langchain_core.messages import HumanMessage

# 메시지 형식 변경 (agent_executor에 맞게)
messages = [
    HumanMessage(query),
]

# agent_executor를 사용하여 실행
response = agent_executor.invoke({"input": messages})
response



> Entering new AgentExecutor chain...

Invoking: `get_web_search` with `{'query': '2025년 KT 소액결제 해킹 사건 원인'}`



----- WEB SEARCH -----
2025년 KT 소액결제 해킹 사건 원인
w
1. snippet: KT 위즈 구단은 25일 2025 시즌 연봉 재계약 대상자 64명과의 계약을 마무리했다고 공식적으로 발표했다., title: KT 위즈 구단은 25일 2025시즌 연봉 재계약 대상자, link: https://totosite.one/2025/01/the-kt-wiz-organization-officially-announced-today-that-it-has-finalized-contracts-with-64-players-for-the-2025-season/
2. snippet: KT Foiling dévoile sa gamme 2025 avec des shapes totalement repensés et cinq nouvelles constructions hautes performances : Carbon, Pro Carbon ..., title: KT Foiling Boards 2025 – Nouvelle génération de planches, link: https://blog.swelladdiction.com/2025/04/20/planches-foil-kt-foiling-2025/
3. snippet: Home › Concerts › United Kingdom › London › KT Tunstall - Royal Albert Hall - Jun 23, 2025 ... for KT Tunstall in London, Jun 23, 2025, title: KT Tunstall, Royal Albert Hall, Jun 23, 2025 Tickets, London,, link: https://www.jambase.com/show/kt-tunstall

{'input': [HumanMessage(content='2025년 KT 소액결제 해킹 사건 원인', additional_kwargs={}, response_metadata={})],
 'output': '2025년 KT 소액결제 해킹 사건에 대한 직접적인 정보를 찾을 수 없습니다. 하지만 KT Wiz 야구단의 연봉 재계약, KT Foiling Boards 2025 신제품, KT Tunstall의 런던 공연 등 다른 정보들이 검색 결과에 나타납니다. 좀 더 구체적인 검색을 위해 다른 키워드를 사용하거나, 검색 기간을 좁혀보는 것을 추천합니다.\n'}